In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm
from langchain_openai import AzureOpenAIEmbeddings
import json
import numpy as np

/Users/olliglorioso/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/olliglorioso/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's fetch the dataframes.

In [2]:
from data_extractor import Extractor
extractor_instance = Extractor()
extractor_instance.extract(location="./data/results/df_combined.csv")

df_combined = pd.read_csv("data/results/df_combined.csv")
df_combined.sample(n=3)

Comp domain df length: 1912  - VTT domain df length: 4927
Example of text to compare: FiR 1 - FiR 1 is a Triga-type nuclear research reactor located in Otaniemi, Espoo, Finland, decommissioned by VTT after serving since 1962, used for nuclear research, training, and medical radiation therapy. | Developed by VTT Technical Research Centre of Finland Ltd.


,Unnamed: 0.1,Document number,VAT id,relationship description,source id,source type,source english_id,source description,relationship type,target id,target type,target english_id,target description,Link Source Text,Source Text,text_to_compare,Unnamed: 0,Source Company
3221,3221,VTT822,FI04205273,"Natlink Oy, owning the Tracker brand, develope...",cyber-physical hunting safety solutions,Innovation,cyber-physical hunting safety solutions,Innovative solutions developed by Natlink Oy a...,DEVELOPED_BY,FI04205273,Organization,Natlink Oy,Natlink Oy is an organization owning the Track...,https://www.vttresearch.com/fi/uutiset-ja-tari...,Hyppää pääsisältöön Beyond the obvious ...,cyber-physical hunting safety solutions - Inno...,NaN,NaN
2918,2918,VTT747,FI06259056,The EcoSMR innovation was developed with parti...,EcoSMR,Innovation,EcoSMR,A Finnish ecosystem for Small Modular Reactors...,DEVELOPED_BY,FI01966560,Organization,Teollisuuden Voima Oyj,A Finnish energy company and a business partne...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,EcoSMR - A Finnish ecosystem for Small Modular...,NaN,NaN
5581,5581,COMP441,NaN,The PV-KK-S blast valve is developed and offer...,PV-KK-S blast valve,Innovation,PV-KK-S blast valve,An application of the PV-KK blast valve design...,DEVELOPED_BY,FI22223934,Organization,Halton Marine,A company that offers the PV-KK-S blast valve ...,https://www.halton.com/wp-content/uploads/2020...,79\nBLAST VALVE â¢ PV-KK-SPV-KK-S \nAPPLIC...,PV-KK-S blast valve - An application of the PV...,2493.0,HALTON VENTURES OY


In [3]:
from embedding_generator import EmbeddingGenerator, LocalEmbeddingProvider, OpenAIEmbeddingProvider

# Configuration
METHOD = "openai"  # Supports local and openai
MODEL_NAME = "all-mpnet-base-v2"  # For local embeddings
MODEL_KEY = "gpt-4.1-mini"  # For OpenAI embeddings, I think only this works

# File paths
input_file = "data/results/df_combined.csv"
output_file = "data/results/embeddings.csv"

provider = OpenAIEmbeddingProvider(MODEL_KEY) if METHOD == "openai" else LocalEmbeddingProvider(MODEL_NAME)
generator = EmbeddingGenerator(provider)

result_df = generator.generate_embeddings(input_file, output_file)

2025-05-31 14:27:06,148 - INFO - Initialized OpenAI embedding model with key: gpt-4.1-mini
2025-05-31 14:27:06,148 - INFO - Loading data from data/results/df_combined.csv
2025-05-31 14:27:06,496 - INFO - Loaded 6839 rows from input file
2025-05-31 14:27:06,497 - INFO - Prepared 6839 texts for embedding generation
2025-05-31 14:27:06,497 - INFO - Generating embeddings for 6839 texts using OpenAI API
Processing batches:  72%|███████▏  | 50/69 [00:50<00:17,  1.11it/s]2025-05-31 14:27:57,548 - INFO - HTTP Request: POST https://aaltoaihack25-resource.cognitiveservices.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 429 Too Many Requests"
2025-05-31 14:27:57,550 - INFO - Retrying request to /embeddings in 14.000000 seconds
2025-05-31 14:28:12,109 - INFO - HTTP Request: POST https://aaltoaihack25-resource.cognitiveservices.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
Proc

In [7]:
result_df.head()

,Unnamed: 0.1,Document number,VAT id,relationship description,source id,source type,source english_id,source description,relationship type,target id,...,target english_id,target description,Link Source Text,Source Text,text_to_compare,Unnamed: 0,Source Company,embedding,embedding_provider,embedding_dimension
0,0,VTT0,FI10292588,"FiR 1 nuclear research reactor was developed, ...",FiR 1,Innovation,FiR 1,FiR 1 is a Triga-type nuclear research reactor...,DEVELOPED_BY,FI26473754,...,VTT Technical Research Centre of Finland Ltd.,VTT is a Finnish research and innovation partn...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,FiR 1 - FiR 1 is a Triga-type nuclear research...,NaN,NaN,"[-0.018910497426986694, 0.017533520236611366, ...",openai_gpt-4.1-mini,3072
1,1,VTT0,FI10292588,Centre for Nuclear Safety is being developed a...,Centre for Nuclear Safety,Innovation,Centre for Nuclear Safety,A modern research facility under construction ...,DEVELOPED_BY,FI26473754,...,VTT Technical Research Centre of Finland Ltd.,VTT is a Finnish research and innovation partn...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,Centre for Nuclear Safety - A modern research ...,NaN,NaN,"[-0.009751166217029095, 0.006150579079985619, ...",openai_gpt-4.1-mini,3072
2,2,VTT3,FI08932048,The innovation approach 'Beyond the obvious' i...,Beyond the obvious,Innovation,Beyond the obvious,An innovation approach promising to provide so...,DEVELOPED_BY,FI26473754,...,VTT Technical Research Centre of Finland Ltd,A visionary research and innovation partner fo...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,Beyond the obvious - An innovation approach pr...,NaN,NaN,"[-0.02210899069905281, -0.002120873425155878, ...",openai_gpt-4.1-mini,3072
3,3,VTT4,FI01111693,Data-Driven Bioeconomy project is developed by...,Data-Driven Bioeconomy project,Innovation,Data-Driven Bioeconomy project,An innovation using Big Data for sustainable u...,DEVELOPED_BY,FI26473754,...,VTT Technical Research Centre of Finland,A Finnish research and innovation partner work...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,Data-Driven Bioeconomy project - An innovation...,NaN,NaN,"[-0.015033945441246033, 0.031899139285087585, ...",openai_gpt-4.1-mini,3072
4,4,VTT4,FI01111693,Data-Driven Bioeconomy project's forestry pilo...,Data-Driven Bioeconomy project,Innovation,Data-Driven Bioeconomy project,An innovation using Big Data for sustainable u...,DEVELOPED_BY,temp_1141,...,MHG Systems,An organization leading pilots developing fore...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...,Data-Driven Bioeconomy project - An innovation...,NaN,NaN,"[-0.014490003697574139, 0.016991155222058296, ...",openai_gpt-4.1-mini,3072


In [9]:
threshold = 0.80
similar_pairs = []

combined_embeddings = result_df["embedding"].tolist()
combined_texts = result_df["text_to_compare"].tolist()

print("Comparing entries pair by pair...")

for i in tqdm(range(len(combined_embeddings))):
    for j in range(i + 1, len(combined_embeddings)):
        sim = 0
        if result_df.iloc[i]["source description"] != result_df.iloc[j]["source description"]:
            sim = 1
        else: 
            sim = util.cos_sim(combined_embeddings[i], combined_embeddings[j]).item()
        
        if sim > threshold:
            i_elem = combined_texts[i]
            j_elem = combined_texts[j]
            
            #if sim < 1.0:
             #   print(f"\nSimilarity: {sim:.3f} | From: {i+1} To: {j+1}")
              #  print(f"→ Source {i+1} description: {i_elem}")
               # print(f"→ Source {j+1} description: {j_elem}")
            
            
            similar_pairs.append((i, j, sim))

# Output results
print(f"\nFound {len(similar_pairs)} high-similarity pairs (> {threshold}):")
for i, j, score in similar_pairs:
    print(f"[{i}] '{df.iloc[i]['source english_id']}' <--> [{j}] '{df.iloc[j]['source english_id']}' | Similarity: {score:.3f}")

Comparing entries pair by pair...


  0%|          | 3/6839 [00:00<38:09,  2.99it/s]


Similarity: 0.879 | From: 4 To: 5
→ Source 4 description: Data-Driven Bioeconomy project - An innovation using Big Data for sustainable use of resources in agriculture, forestry and fishery, improving decision-making with sensors and imagery data. | Developed by VTT Technical Research Centre of Finland
→ Source 5 description: Data-Driven Bioeconomy project - An innovation using Big Data for sustainable use of resources in agriculture, forestry and fishery, improving decision-making with sensors and imagery data. | Developed by MHG Systems


  0%|          | 5/6839 [00:01<34:16,  3.32it/s]


Similarity: 0.962 | From: 6 To: 7
→ Source 6 description: Alternatives to toxic phenol compounds used in glues are being developed from softwood tannins - A novel industrial scale method and products for renewable glue resins derived from softwood bark tannin-polyphenols as alternatives to fossil and toxic phenol compounds used in glues. | Developed by VTT Technical Research Centre of Finland
→ Source 7 description: Alternatives to toxic phenol compounds used in glues are being developed from softwood tannins - A novel industrial scale method and products for renewable glue resins derived from softwood bark tannin-polyphenols as alternatives to fossil and toxic phenol compounds used in glues. | Developed by Natural Resources Institute Finland (Luke)

Similarity: 0.962 | From: 6 To: 8
→ Source 6 description: Alternatives to toxic phenol compounds used in glues are being developed from softwood tannins - A novel industrial scale method and products for renewable glue resins derived from

...


Similarity: 0.899 | From: 90 To: 91
→ Source 90 description: Anti-theft sticker - A prototype adhesive ID tag based on blockchain technology that protects valuables without revealing their location. | Developed by VTT Technical Research Centre of Finland
→ Source 91 description: Anti-theft sticker - A prototype adhesive ID tag based on blockchain technology that protects valuables without revealing their location. | Developed by Streamr


  1%|▏         | 91/6839 [00:25<30:54,  3.64it/s]


Similarity: 0.918 | From: 92 To: 93
→ Source 92 description: Anti-theft sticker prototype based on blockchain technology - A prototype adhesive ID tag developed to protect valuables without revealing their location, using blockchain technology and smart contracts. | Developed by VTT Technical Research Centre of Finland
→ Source 93 description: Anti-theft sticker prototype based on blockchain technology - A prototype adhesive ID tag developed to protect valuables without revealing their location, using blockchain technology and smart contracts. | Developed by Streamr


  1%|▏         | 93/6839 [00:25<30:57,  3.63it/s]


KeyboardInterrupt: 